In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("./data/clean_data_merged_v1.parquet")
data['HomeTeamName'] = data['HomeTeamName'].str.strip()
data['AwayTeamName'] = data['AwayTeamName'].str.strip()

#data = data[data['date'].dt.year >= 1980].copy()
data['year'] = data['date'].dt.year

In [3]:
data.head(5)

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year
0,1960-07-06 20:00:00,France,Yugoslavia,4,5,-1.0,-1,1960
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3,-1.0,-3,1960
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0,1.0,2,1960
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1,1.0,1,1960
4,1964-06-17 20:00:00,Spain,Hungary,2,1,1.0,1,1964


In [4]:
data['won'] = (data['HomeTeamGoals'] > data['AwayTeamGoals']).astype(int)
data['lost'] = (data['HomeTeamGoals'] < data['AwayTeamGoals']).astype(int)
data['tie'] = (data['HomeTeamGoals'] == data['AwayTeamGoals']).astype(int)


In [5]:
gen_stats = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_stats.columns = ['n_games', 'home_goals', 'away_goals', 'won', 'lost', 'tie']

In [6]:
gen_statsh = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsh.columns = ['n_games', 'home_goals_scored', 'away_goals_suffered', 'won', 'lost', 'tie']
gen_statsh = gen_statsh.reset_index()
gen_statsh

,HomeTeamName,year,n_games,home_goals_scored,away_goals_suffered,won,lost,tie
0,Albania,2016,1,0,1,0,1,0
1,Austria,2008,3,1,3,0,2,1
2,Austria,2016,1,0,2,0,1,0
3,Belgium,1972,1,1,2,0,1,0
4,Belgium,1980,3,4,4,1,1,1
...,...,...,...,...,...,...,...,...
141,West Germany,1980,1,3,2,1,0,0
142,West Germany,1984,3,2,2,1,1,1
143,West Germany,1988,4,6,3,2,1,1
144,Yugoslavia,1968,1,1,0,1,0,0


In [7]:
gen_statsh = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsh.columns = ['n_games', 'scored_goals', 'suffered_goals', 'won', 'lost', 'tie']
gen_statsh.index = gen_statsh.index.rename(['team', 'year'])


gen_statsa = data.groupby(["AwayTeamName", "year"]).agg({"AwayTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsa.columns = ['n_games', 'suffered_goals', 'scored_goals', 'won', 'lost', 'tie']
gen_statsa.index = gen_statsa.index.rename(['team', 'year'])

gen_stats = gen_statsh.add(gen_statsa, fill_value=0).reset_index()
gen_stats.sort_values("year", inplace=True)


gen_stats['year'] += 1

# stats um time contra o outro

In [10]:
gen_stats

,team,year,lost,n_games,scored_goals,suffered_goals,tie,won
152,Yugoslavia,1961,1.0,2.0,6.0,6.0,0.0,1.0
115,Soviet Union,1961,1.0,2.0,5.0,1.0,0.0,1.0
43,France,1961,1.0,2.0,4.0,7.0,0.0,1.0
22,Czechoslovakia,1961,1.0,2.0,2.0,3.0,0.0,1.0
63,Hungary,1965,0.0,2.0,4.0,3.0,0.0,2.0
...,...,...,...,...,...,...,...,...
100,Republic of Ireland,2017,1.0,4.0,3.0,6.0,1.0,2.0
86,Northern Ireland,2017,2.0,4.0,2.0,3.0,0.0,2.0
97,Portugal,2017,1.0,7.0,9.0,5.0,4.0,2.0
51,France,2017,1.0,7.0,13.0,5.0,1.0,5.0


In [34]:
gen_stats_cumulative = gen_stats.groupby("team").cumsum()
gen_stats_cumulative['team'] = gen_stats['team']
gen_stats_cumulative['year'] = gen_stats['year']
gen_stats_cumulative.sort_values("year", inplace=True)
gen_stats_cumulative

,year,lost,n_games,scored_goals,suffered_goals,tie,won,team
152,1961,1.0,2.0,6.0,6.0,0.0,1.0,Yugoslavia
115,1961,1.0,2.0,5.0,1.0,0.0,1.0,Soviet Union
43,1961,1.0,2.0,4.0,7.0,0.0,1.0,France
22,1961,1.0,2.0,2.0,3.0,0.0,1.0,Czechoslovakia
63,1965,0.0,2.0,4.0,3.0,0.0,2.0,Hungary
...,...,...,...,...,...,...,...,...
139,2017,5.0,13.0,8.0,15.0,5.0,3.0,Switzerland
143,2017,7.0,15.0,13.0,22.0,2.0,6.0,Turkey
51,2017,11.0,39.0,62.0,44.0,9.0,19.0,France
129,2017,14.0,40.0,55.0,36.0,11.0,15.0,Spain


In [35]:
gen_stats[gen_stats['team'] == 'Portugal']

,team,year,lost,n_games,scored_goals,suffered_goals,tie,won
91,Portugal,1985,0.0,4.0,4.0,4.0,2.0,2.0
92,Portugal,1997,1.0,4.0,5.0,2.0,1.0,2.0
93,Portugal,2001,2.0,5.0,10.0,4.0,0.0,3.0
94,Portugal,2005,4.0,6.0,8.0,6.0,1.0,1.0
95,Portugal,2009,2.0,4.0,7.0,6.0,0.0,2.0
96,Portugal,2013,2.0,5.0,6.0,4.0,1.0,2.0
97,Portugal,2017,1.0,7.0,9.0,5.0,4.0,2.0


In [36]:
gen_stats_cumulative[gen_stats_cumulative['team'] == 'Portugal']

,year,lost,n_games,scored_goals,suffered_goals,tie,won,team
91,1985,0.0,4.0,4.0,4.0,2.0,2.0,Portugal
92,1997,1.0,8.0,9.0,6.0,3.0,4.0,Portugal
93,2001,3.0,13.0,19.0,10.0,3.0,7.0,Portugal
94,2005,7.0,19.0,27.0,16.0,4.0,8.0,Portugal
95,2009,9.0,23.0,34.0,22.0,4.0,10.0,Portugal
96,2013,11.0,28.0,40.0,26.0,5.0,12.0,Portugal
97,2017,12.0,35.0,49.0,31.0,9.0,14.0,Portugal


In [37]:
def fn_features(df, team_features):
    df_ = pd.merge_asof(df, team_features, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home', '_away'])
    
    return df_

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss, roc_auc_score

In [44]:
for year in [2004, 2008, 2012, 2016]:
    tr = data[data['year'] < year].copy()
    val = data[data['year'] == year].copy()
    
    tr = tr.drop(['won', 'lost', 'tie'], axis=1)
    val = val.drop(['won', 'lost', 'tie'], axis=1)
    
   
    selected_features = ['lost_home',
       'n_games_home', 'scored_goals_home', 'suffered_goals_home', 'tie_home',
       'won_home', 'lost_away', 'n_games_away',
       'scored_goals_away', 'suffered_goals_away', 'tie_away', 'won_away']
    
    tr_ = fn_features(tr, gen_stats_cumulative)
    val_ = fn_features(val, gen_stats_cumulative)
    
    
    Xtr = tr_[selected_features].fillna(0)
    Xval = val_[selected_features].fillna(0)
    ytr = tr['y_classif']
    yval = val['y_classif']
    
    mdl = LogisticRegression()
    mdl.fit(Xtr, ytr)
    
    p = mdl.predict_proba(Xval)
    
    
        
    baseline = ytr.value_counts(normalize=True)
    baseline.index = [2,0,1]
    baseline = np.ones((Xval.shape[0], 3)) * baseline.sort_index().values
    
    
    print("Ano: {} | Baseline Log Loss: {:.4f} | Baseline AUC: {:.4f} | Log Loss: {:.4f} | ROC AUC: {:.4f}\n".format(year, log_loss(yval, baseline),
            roc_auc_score(yval, baseline, multi_class='ovr'), log_loss(yval, p), roc_auc_score(yval,p, multi_class='ovr')))
    #break


Ano: 2004 | Baseline Log Loss: 1.1082 | Baseline AUC: 0.5000 | Log Loss: 1.3765 | ROC AUC: 0.5346

Ano: 2008 | Baseline Log Loss: 1.0736 | Baseline AUC: 0.5000 | Log Loss: 1.2565 | ROC AUC: 0.5599

Ano: 2012 | Baseline Log Loss: 1.0708 | Baseline AUC: 0.5000 | Log Loss: 1.1822 | ROC AUC: 0.5477

Ano: 2016 | Baseline Log Loss: 1.0898 | Baseline AUC: 0.5000 | Log Loss: 1.1091 | ROC AUC: 0.6031



/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

2004 Baseline Home Win 0.3548387096774194
              precision    recall  f1-score   support

        -1.0       0.19      0.30      0.23        10
         0.0       0.00      0.00      0.00        10
         1.0       0.33      0.45      0.38        11

    accuracy                           0.26        31
   macro avg       0.17      0.25      0.21        31
weighted avg       0.18      0.26      0.21        31

2004               precision    recall  f1-score   support

        -1.0       0.27      0.40      0.32        10
         0.0       0.00      0.00      0.00        10
         1.0       0.44      0.64      0.52        11

    accuracy                           0.35        31
   macro avg       0.23      0.35      0.28        31
weighted avg       0.24      0.35      0.29        31



2008 Baseline Historical wins 0.41935483870967744
2008 Baseline Home Win 0.3548387096774194
              precision    recall  f1-score   support

        -1.0       0.47      0.53      0.50        15
         0.0       0.00      0.00      0.00         5
         1.0       0.36      0.45      0.40        11

    accuracy                           0.42        31
   macro avg       0.28      0.33      0.30        31
weighted avg       0.35      0.42      0.38        31


2008               precision    recall  f1-score   support

        -1.0       0.62      0.53      0.57        15
         0.0       0.17      0.20      0.18         5
         1.0       0.42      0.45      0.43        11

    accuracy                           0.45        31
   macro avg       0.40      0.40      0.40        31
weighted avg       0.47      0.45      0.46        31


2012 Baseline Historical wins 0.2903225806451613
2012 Baseline Home Win 0.41935483870967744
              precision    recall  f1-score   support

        -1.0       0.30      0.55      0.39        11
         0.0       0.00      0.00      0.00         7
         1.0       0.27      0.23      0.25        13

    accuracy                           0.29        31
   macro avg       0.19      0.26      0.21        31
weighted avg       0.22      0.29      0.24        31

2012               precision    recall  f1-score   support

        -1.0       0.41      0.82      0.55        11
         0.0       0.00      0.00      0.00         7
         1.0       0.67      0.46      0.55        13

    accuracy                           0.48        31
   macro avg       0.36      0.43      0.36        31
weighted avg       0.42      0.48      0.42        31


2016 Baseline Historical wins 0.4117647058823529
2016 Baseline Home Win 0.39215686274509803
              precision    recall  f1-score   support

        -1.0       0.36      0.76      0.49        17
         0.0       0.00      0.00      0.00        14
         1.0       0.53      0.40      0.46        20

    accuracy                           0.41        51
   macro avg       0.30      0.39      0.32        51
weighted avg       0.33      0.41      0.34        51


2016               precision    recall  f1-score   support

        -1.0       0.42      0.47      0.44        17
         0.0       0.00      0.00      0.00        14
         1.0       0.47      0.75      0.58        20

    accuracy                           0.45        51
   macro avg       0.30      0.41      0.34        51
weighted avg       0.32      0.45      0.37        51

In [104]:
val_

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year,team_home,n_games_home,cumulative_games_home,team_away,n_games_away,cumulative_games_away
0,2004-06-12 17:00:00,Portugal,Greece,1,2,-1.0,-1,2004,Portugal,5.0,13.0,Greece,3.0,3.0
1,2004-06-12 19:45:00,Spain,Russia,1,0,1.0,1,2004,Spain,4.0,21.0,Russia,3.0,3.0
2,2004-06-16 17:00:00,Greece,Spain,1,1,0.0,0,2004,Greece,3.0,3.0,Spain,4.0,21.0
3,2004-06-16 19:45:00,Russia,Portugal,0,2,-1.0,-2,2004,Russia,3.0,3.0,Portugal,5.0,13.0
4,2004-06-20 19:45:00,Spain,Portugal,0,1,-1.0,-1,2004,Spain,4.0,21.0,Portugal,5.0,13.0
5,2004-06-20 19:45:00,Russia,Greece,2,1,1.0,1,2004,Russia,3.0,3.0,Greece,3.0,3.0
6,2004-06-13 17:00:00,Switzerland,Croatia,0,0,0.0,0,2004,Switzerland,3.0,3.0,Croatia,4.0,4.0
7,2004-06-13 19:45:00,France,England,2,1,1.0,1,2004,France,6.0,21.0,England,3.0,19.0
8,2004-06-17 17:00:00,England,Switzerland,3,0,1.0,3,2004,England,3.0,19.0,Switzerland,3.0,3.0
9,2004-06-17 19:45:00,Croatia,France,2,2,0.0,0,2004,Croatia,4.0,4.0,France,6.0,21.0
